In [1]:
import numpy as np
import pandas as pd
from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import lightgbm as lgbm
import warnings
warnings.filterwarnings('ignore')

C:\Users\samsung\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# 깨달음
1. 모든 변수를 일일히 다 하나씩 빼놓고 점검하는 건, 시간이 오래~~ 걸린다. <br/>
    - 앞으론 Correlation 적극적으로 활용하자
    - 만약 모든 변수를 하나씩 점검한다면, 잠자기 전에 돌리고 다음 날 확인하자
2. 막 get_dummies 돌리지말자.<br/>
    - 케이스가 2개밖에 안되는데, get_dummies를 돌려서 똑같은 데이터를 두번 점검한 경우가 있었다.
3. ligthGBM predict할 때 raw data를 사용하자. lgbm.DataSet 사용하지 말고<br/>
<br/>

# Delete Unnecessary Variables with LightGBM

--Test1--</br>
 1. Insert all variables except one variables</br>
 2. Extract unnecessary variables</br>
 3. Give back necessary varaibles to preprocess file</br>

### 1. Delete Candidate
    - whour, passenger_count
    - several condition useless
### 2. Important Candidate
    - hour, wday, dropoff_latitude, distance_jfk_pick
    - total_travel_time ,...


In [59]:
df.sort_values('RMSE').head(10).T

,10,1,36,35,34,32,33,37,38,24
variable,whour,passenger_count,c_Unknown,c_Snow,c_Scattered Clouds,c_Partly Cloudy,c_Rain,sf_N,sf_Y,c_Haze
RMSE,0.348208,0.348275,0.348465,0.348466,0.348466,0.348466,0.348466,0.348521,0.348521,0.348551


In [60]:
df.sort_values('RMSE').tail(10).T

,4,20,17,22,7,21,16,5,8,9
variable,dropoff_longitude,total_distance,distance_jfk_drop,number_of_steps,yday,total_travel_time,distance_jfk_pick,dropoff_latitude,wday,hour
RMSE,0.349717,0.349827,0.350047,0.350113,0.350405,0.350515,0.350912,0.351091,0.359471,0.374061


In [2]:
train = pd.read_csv('train_set2.csv')
test = pd.read_csv('test_set2.csv')

In [33]:
t_train = train.copy()
t_test = test.copy()

In [ ]:
kf = KFold(train.shape[0], n_folds = 3,  random_state=1000)            

for (train_index, cross_index) in kf:
    t_train, _ = train.iloc[train_index], train.iloc[cross_index]

t_train.index = pd.RangeIndex(t_train.shape[0])
t_test = test.copy()

In [34]:
t_train.drop(['id','pickup_datetime', 'dropoff_datetime'], axis = 1, inplace = True)
t_test.drop(['id','pickup_datetime'], axis = 1, inplace = True)
ix = 0
for df in [t_train, t_test]:
    df_condition = pd.get_dummies(df['condition'], prefix = 'c', prefix_sep = '_')
    df.drop(['condition'], axis =1, inplace= True)
    df_Master = pd.concat([df, df_condition, df_store_and_fwd_flag], axis = 1)
    if ix == 0: t_train= df_Master
    else: t_test = df_Master
    ix += 1

In [52]:
def lgbmExceptOneCol(t_train, t_test):
    t_train['trip_duration'] = np.log(t_train['trip_duration'])
    Y_train = t_train['trip_duration'].copy()
    t_train = t_train.drop(['trip_duration'], axis =1)
    lgbm_rounds = []
    NFOLDS = 5
    cv_score_list = []
    #diction = ('vendor_id','passenger_count', 'pickup_longitude', 'pickup_atitude', 'dropoff_longitude', 
#'dropoff_latitude', 'month', 'yday', 'wday', 'hour', 'whour')
    lgbm_params = {
    'boosting_type': 'gbdt', 'objective': 'regression', 'nthread': -1, 
    'num_leaves': 32, 'learning_rate': 0.03, 'max_depth': -1,
    'max_bin': 255, 'subsample_for_bin': 50000, 'metric': 'l2_root',
    'subsample': 0.8, 'subsample_freq': 1, 'colsample_bytree': 0.7, 'reg_alpha': 1, 'reg_lambda': 1.2,
    'min_split_gain': 0.6, 'min_child_weight': 4, 'min_child_samples': 10, 'scale_pos_weight': 1}

    kf = KFold(t_train.shape[0], n_folds = NFOLDS,  random_state=1000)
    colList = t_train.columns
    res = []
    for col in colList:
        #if col == 'c_Light Freezing Fog': continue
        #if col in diction: continue
        print(col)
        cv_sum = 0
        for ix, (train_index, cross_index) in enumerate(kf):
            #print("{} Fold \n".format(ix))
            train = t_train.drop(col, axis = 1)
            test = t_test.drop(col, axis = 1)
            x_train, x_cross = train.iloc[train_index], train.iloc[cross_index]
            y_train, y_cross = Y_train.iloc[train_index], Y_train.iloc[cross_index]
            dtrain = lgbm.Dataset(x_train, label = y_train.values, silent = True)
            dvalid = lgbm.Dataset(x_cross, label = y_cross.values, silent = True)
            model = lgbm.train(lgbm_params, train_set = dtrain, num_boost_round = 200, valid_sets = dvalid,
                      early_stopping_rounds = 100, verbose_eval = None)
            lgbm_rounds.append(model.best_iteration)
            scores_val = model.predict(x_cross, num_iteration = model.best_iteration)
            cv_score = sqrt(mean_squared_error(y_cross, scores_val))
            #print('eval-MAE : %.6f' % cv_score)
            cv_score_list.append(cv_score)
            cv_sum = cv_sum + cv_score
    
        score = cv_sum / NFOLDS
        print("Average LogLikelihood: ", score)
        res.append([col, score])
                
    return res

In [9]:
colRMSE = lgbmExceptOneCol(t_train, t_test)

pickup_latitude
Average LogLikelihood:  0.34864094929406997
pickup_cluster
Average LogLikelihood:  0.34860272373145307
dropoff_cluster
Average LogLikelihood:  0.3486862380300073
distance_haversine
Average LogLikelihood:  0.3494901553541262
distance_dummy_manhattan
Average LogLikelihood:  0.34866986059946176
distance_jfk_pick
Average LogLikelihood:  0.3509120254975151
distance_jfk_drop
Average LogLikelihood:  0.35004673759299215
distance_laguardia_pick
Average LogLikelihood:  0.3486718461809466
distance_laguardia_drop
Average LogLikelihood:  0.3486915759595598
total_distance
Average LogLikelihood:  0.34982652133062947
total_travel_time
Average LogLikelihood:  0.35051478468174374
number_of_steps
Average LogLikelihood:  0.35011327124048275
c_Clear
Average LogLikelihood:  0.34856419943551764
c_Haze
Average LogLikelihood:  0.3485509547455716
c_Heavy Rain
Average LogLikelihood:  0.3485509547455716
c_Heavy Snow
Average LogLikelihood:  0.3485509547455716
c_Light Freezing Rain
Average LogLikeli

In [11]:
colRMSE2 = [['vendor_id',0.3486582532098187],
['passenger_count', 0.6969328370425323],
['pickup_longitude', 1.0458056055466818],
['pickup_atitude', 1.3944465548407519],
['dropoff_longitude', 1.744163925776532],
['dropoff_latitude', 2.0952547811750097],
['month', 2.4438334138834468],
['yday', 2.794238195924415],
['wday',  3.1537092325381457],
['hour', 3.5277706703579073],
['whour',3.875978409090443]]
for itr in reversed(range(len(colRMSE2))):
    if itr == 0: continue
    colRMSE2[itr][1] -= colRMSE2[itr-1][1]

In [15]:
colRMSE2.extend(colRMSE)

In [53]:
df = pd.DataFrame(colRMSE2, columns = ['variable', 'RMSE'])

### 1. Delete Candidate
    - whour, passenger_count
    - several condition useless


In [57]:
df.sort_values('RMSE').head(10).T

,10,1,36,35,34,32,33,37,38,24
variable,whour,passenger_count,c_Unknown,c_Snow,c_Scattered Clouds,c_Partly Cloudy,c_Rain,sf_N,sf_Y,c_Haze
RMSE,0.348208,0.348275,0.348465,0.348466,0.348466,0.348466,0.348466,0.348521,0.348521,0.348551


### 2. Important Candidate
    - hour, wday, dropoff_latitude, distance_jfk_pick
    - total_travel_time ,...

In [58]:
df.sort_values('RMSE').tail(10).T

,4,20,17,22,7,21,16,5,8,9
variable,dropoff_longitude,total_distance,distance_jfk_drop,number_of_steps,yday,total_travel_time,distance_jfk_pick,dropoff_latitude,wday,hour
RMSE,0.349717,0.349827,0.350047,0.350113,0.350405,0.350515,0.350912,0.351091,0.359471,0.374061


In [16]:
colRMSE2

[['vendor_id', 0.3486582532098187],
 ['passenger_count', 0.3482745838327136],
 ['pickup_longitude', 0.3488727685041495],
 ['pickup_atitude', 0.3486409492940701],
 ['dropoff_longitude', 0.34971737093578015],
 ['dropoff_latitude', 0.35109085539847773],
 ['month', 0.34857863270843703],
 ['yday', 0.35040478204096814],
 ['wday', 0.3594710366137308],
 ['hour', 0.37406143781976153],
 ['whour', 0.34820773873253597],
 ['pickup_latitude', 0.34864094929406997],
 ['pickup_cluster', 0.34860272373145307],
 ['dropoff_cluster', 0.3486862380300073],
 ['distance_haversine', 0.3494901553541262],
 ['distance_dummy_manhattan', 0.34866986059946176],
 ['distance_jfk_pick', 0.3509120254975151],
 ['distance_jfk_drop', 0.35004673759299215],
 ['distance_laguardia_pick', 0.3486718461809466],
 ['distance_laguardia_drop', 0.3486915759595598],
 ['total_distance', 0.34982652133062947],
 ['total_travel_time', 0.35051478468174374],
 ['number_of_steps', 0.35011327124048275],
 ['c_Clear', 0.34856419943551764],
 ['c_Haze'

In [51]:
mdl, score = lgbmTest(t_train, t_test)

0 Fold 

eval-MAE : 0.035628
1 Fold 

eval-MAE : 0.035214
2 Fold 

eval-MAE : 0.035302
3 Fold 

eval-MAE : 0.035578
4 Fold 

eval-MAE : 0.035430
Average LogLikelihood:  0.03543057645141947
1 0.03562797407216023
2 0.03521445493252553
3 0.035301523287573654
4 0.03557846155301239
5 0.035430468411825546
test prediction
bst Round : 203


In [62]:
mdl.predict(t_test)

array([ 0.33980354,  0.33340087,  0.31651811, ...,  0.32490772,
        0.32973411,  0.32973411])

In [50]:
#train_file = "NYC/train_set1.csv"
#test_file = "NYC/test_set1.csv"
def lgbmTest(train, test):
    train['trip_duration'] = np.log(train['trip_duration'])
    Y_train = train['trip_duration'].copy()
    train = train.drop(['trip_duration'], axis =1)
    cv_sum = 0
    fpred = []
    lgbm_rounds = []
    NFOLDS = 5
    cv_score_list = []
    lgbm_params = {
    'boosting_type': 'gbdt', 'objective': 'regression', 'nthread': -1,
    'num_leaves': 32, 'learning_rate': 0.03, 'max_depth': -1,
    'max_bin': 255, 'subsample_for_bin': 50000, 'metric': 'l2_root',
    'subsample': 0.8, 'subsample_freq': 1, 'colsample_bytree': 0.7, 'reg_alpha': 1, 'reg_lambda': 1.2,
    'min_split_gain': 0.6, 'min_child_weight': 4, 'min_child_samples': 10, 'scale_pos_weight': 1}

    kf = KFold(train.shape[0], n_folds = NFOLDS)
    for ix, (train_index, cross_index) in enumerate(kf):
        print("{} Fold \n".format(ix))
        x_train, x_cross = train.iloc[train_index], train.iloc[cross_index]
        y_train, y_cross = Y_train.iloc[train_index], Y_train.iloc[cross_index]
    
        dtrain = lgbm.Dataset(x_train, label = y_train.values, silent = True)
        dvalid = lgbm.Dataset(x_cross, label = y_cross.values, silent = True)
        model = lgbm.train(lgbm_params, train_set = dtrain, num_boost_round = 100000, valid_sets = dvalid,
                      early_stopping_rounds = 100, verbose_eval = None,)
        lgbm_rounds.append(model.best_iteration)
        scores_val = model.predict(x_cross, num_iteration = model.best_iteration)
        cv_score = sqrt(mean_squared_error(y_cross, scores_val))
        print('eval-MAE : %.6f' % cv_score)
        cv_score_list.append(cv_score)
        cv_sum = cv_sum + cv_score
    
    score = cv_sum / NFOLDS
    print("Average LogLikelihood: ", score)
    for cv_index, cv_s in enumerate(cv_score_list):
        print(cv_index+1, cv_s)
    
    print('test prediction')
    bstRound = max(lgbm_rounds)
    print('bst Round :', bstRound)
    
    dtrain_all = lgbm.Dataset(train, label = Y_train.values, silent = True)
    model = lgbm.train(lgbm_params, dtrain_all)
    predictions = model.predict(train, num_iteration = bstRound)
    cv_score= sqrt(mean_squared_error(predictions, Y_train.values))
    return model, cv_score
    #dtest = lgbm.Dataset(test.values)
    #model = lgbm.train(lgbm_params, dtrain_all)
    #predictions = model.predict(test, num_iteration = bstRound)
    return predictions
    